In [218]:
import pandas as pd

In [219]:
df_precos_leilao = pd.read_csv("../dados/PrecoLeiloes.csv")

In [220]:
df_precos_leilao.head()

,CNPJ,SiglaVendedor,PrecoVenda,UnidadePreco,UF,InicioSuprimento,AnoLeilao
0,00.357.038/0001-16,ELETRONORTE,212.11,R$/MWh,-,2023-01-01,2021
1,00.622.416/0001-41,STATKRAFT,146.61,R$/MWh,BA,2026-01-01,2021
2,07.674.341/0001-91,UTE Uberaba 2,296.55,R$/MWh,MG,2026-01-01,2021
3,07.912.062/0001-19,LAGUNA,256.00,R$/MWh,MS,2025-01-01,2021
4,07.966.116/0001-29,ALBIOMA,216.97,R$/MWh,GO,2025-01-01,2021


In [221]:
df_info_geradoras = pd.read_csv("../dados/InfoGeradoras.csv")

In [222]:
df_info_geradoras.head()

,CNPJ,Sigla Vendedor,Fonte,NotaSustentabilidade
0,00.357.038/0001-16,ELETRONORTE,-,5.0
1,00.622.416/0001-41,STATKRAFT,Eólica,9.0
2,07.674.341/0001-91,UTE Uberaba 2,Biomassa,8.0
3,07.912.062/0001-19,LAGUNA,Biomassa,8.0
4,07.966.116/0001-29,ALBIOMA,Biomassa,8.0


In [223]:
df_tarifas_distribuidoras = pd.read_csv("../dados/TarifasDistribuidoras.csv")

In [224]:
df_tarifas_distribuidoras.head()

,_id,DatGeracaoConjuntoDados,DscREH,SigAgente,NumCNPJDistribuidora,DatInicioVigencia,DatFimVigencia,DscBaseTarifaria,DscSubGrupo,DscModalidadeTarifaria,DscClasse,DscSubClasse,DscDetalhe,NomPostoTarifario,DscUnidadeTerciaria,SigAgenteAcessante,VlrTUSD,VlrTE,rank
0,67736,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.014, DE 15 DE MAR...",LIGHT,60444437000146,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,463.33,346.57,0.057309
1,70927,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.015, DE 15 DE MAR...",ENEL RJ,33050071000158,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,523.52,307.44,0.057309


## Dados de Geradoras

In [257]:
df_geradoras = df_precos_leilao.merge(df_info_geradoras)[["SiglaVendedor", "PrecoVenda", "NotaSustentabilidade", "Fonte"]]

In [258]:
df_geradoras.head()

,SiglaVendedor,PrecoVenda,NotaSustentabilidade,Fonte
0,ELETRONORTE,212.11,5.0,-
1,STATKRAFT,146.61,9.0,Eólica
2,UTE Uberaba 2,296.55,8.0,Biomassa
3,LAGUNA,256.00,8.0,Biomassa
4,ALBIOMA,216.97,8.0,Biomassa


## Dados Distribuição

In [259]:
df_distribuidoras = df_tarifas_distribuidoras[["SigAgente", "VlrTUSD", "VlrTE"]]

In [260]:
df_tarifas_distribuidoras

,_id,DatGeracaoConjuntoDados,DscREH,SigAgente,NumCNPJDistribuidora,DatInicioVigencia,DatFimVigencia,DscBaseTarifaria,DscSubGrupo,DscModalidadeTarifaria,DscClasse,DscSubClasse,DscDetalhe,NomPostoTarifario,DscUnidadeTerciaria,SigAgenteAcessante,VlrTUSD,VlrTE,rank
0,67736,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.014, DE 15 DE MAR...",LIGHT,60444437000146,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,463.33,346.57,0.057309
1,70927,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.015, DE 15 DE MAR...",ENEL RJ,33050071000158,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,523.52,307.44,0.057309


## Otimização

In [261]:
import cvxpy as cp
import numpy as np

In [262]:
venda = df_geradoras["PrecoVenda"].values
nota = df_geradoras["NotaSustentabilidade"].values
quant_geradoras = len(df_geradoras)

In [263]:
X = cp.Variable((quant_geradoras,1))

In [264]:
gasto_mwh = 270/1000

## Com sustentabilidade

$ \text{minimize } w = Perc * ( venda - k * nota_{sustentabilidade}) $

$ \text{st.}  \sum Perc = gasto_{mwh} $

$ \text{st.}  Perc_i / gasto_{mwh} <= 0.75 $

In [265]:
k = 0

In [266]:
objective = cp.Minimize(cp.sum(cp.multiply(X, [venda - k * nota])))
total_mwh = cp.sum(X) == gasto_mwh
proporcao = X/gasto_mwh <= 0.75
constraints = [X >= 0, total_mwh, proporcao]
prob = cp.Problem(objective, constraints)

In [267]:
result = prob.solve()

In [268]:
df_geradoras["Proposta"] = X.value

In [269]:
df_geradoras.sort_values("Proposta", ascending=False)[:5]

,SiglaVendedor,PrecoVenda,NotaSustentabilidade,Fonte,Proposta
16,SÃO ROQUE ENERGÉTICA,119.30,8.0,Hidrelétrica (UHE),2.025000e-01
17,SOLAR TECNOLOGIA,132.75,9.0,Solar Fotovoltaica,6.750000e-02
36,SANTA LUZIA IX,149.34,9.0,Solar Fotovoltaica,5.439741e-12
35,SANTA LUZIA VII,147.07,9.0,Solar Fotovoltaica,5.315233e-12
1,STATKRAFT,146.61,9.0,Eólica,5.248906e-12


## Com sustentabilidade

$ \text{minimize } w = Perc * ( venda - k * nota_{sustentabilidade}) $

$ \text{st.}  \sum Perc = total_{kwh} $

In [270]:
k = venda.mean() / nota.mean()

In [271]:
objective = cp.Minimize(cp.sum(cp.multiply(X, [venda - k * nota])))
total_mwh = cp.sum(X) == gasto_mwh
proporcao = X/gasto_mwh <= 0.75
constraints = [X >= 0, total_mwh, proporcao]
prob = cp.Problem(objective, constraints)

In [272]:
result = prob.solve()

In [273]:
df_geradoras["Proposta"] = X.value

In [274]:
df_geradoras.sort_values("Proposta", ascending=False)[:5]

,SiglaVendedor,PrecoVenda,NotaSustentabilidade,Fonte,Proposta
17,SOLAR TECNOLOGIA,132.75,9.0,Solar Fotovoltaica,2.025000e-01
37,SANTA LUZIA V,140.93,9.0,Solar Fotovoltaica,6.750000e-02
36,SANTA LUZIA IX,149.34,9.0,Solar Fotovoltaica,2.796731e-12
16,SÃO ROQUE ENERGÉTICA,119.30,8.0,Hidrelétrica (UHE),2.449162e-12
35,SANTA LUZIA VII,147.07,9.0,Solar Fotovoltaica,2.204459e-12
